<a href="https://colab.research.google.com/github/killian31/mva_dlts_24_denoising/blob/test_branch_name/Notebook%20-%20KS%20FS%20-%20Denoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Signal Processing and Deep Learning Final Project : Denoising

Authors : Killian Stennou, François Saulnier

## Revue de litérature
Cette section présente une rapide revue des méthodes proposées pour traiter ce problème, divisée en deux partie : 
- Les méthodes agissant sur le spectrogramme : A. Jansson et Al., SINGING VOICE SEPARATION WITH DEEP U-NET CONVOLUTIONAL NETWORK, ISMIR 2017
- Les méthodes agissant sur la forme d'onde  :
    - D. Stoller  et Al., WAVE-U-NET: A MULTI-SCALE NEURAL NETWORK FOR END-TO-END AUDIO SOURCE SEPARATION, ISMIR 2018
    - TAS NEt :
        - Y. Luo et Al., TaSNet: Time-Domain Audio Separation Network for Real-Time, Single-Channel
    Speech Separation, ICASSP 2018 
        - Y. Luo et Al.,  Conv-tasnet: surpassing ideal time–frequency magnitude masking
    for speech separation. IEEE/ACM Transactions on Audio, Speech, and Language Processing, 2019.

### Méthodes Spectrogramme
#### *A. Jansson et Al., Singing Voice Separation with Deep U-Net Convolutional Network, ISMIR 2017*

##### Idée générale

- Le but est d'isoler la mélodie chantée de l'accompagnement instrumental.
- Deux U-Net : 
    - un pour apprendre à masquer l'accompagnement par le masque $f(X, \Theta_v)$, qui appliqué au spectrogramme $X$ donné au modèle, donne une prédiction de la voix. La loss est $|| f(X, \Theta_v) \odot X - Y_v ||_{1, 1}$
    - un pour apprendre à masquer la voix par le masque $f(X, \Theta_i)$, qui appliqué au spectrogramme $X$ donné au modèle, donne une prédiction des instruments. La loss est $|| f(X, \Theta_i) \odot X - Y_i ||_{1, 1}$

##### Architecture

![Deep U-Net audio source separation architecture](https://camo.githubusercontent.com/ee525acc2add2198749036bf76ea627b9972f881ee844099cb301f9745b843ba/68747470733a2f2f706963342e7a68696d672e636f6d2f76322d38646638636164316466343765346134626537363533373831353636333335325f31323030783530302e6a7067)

##### Évaluation
Pour mesurer la performance, ils utilisent les fonctions suivantes: 
- Normalized Signal-To-Distortion Ratio (NSDR) : $\text{NSDR}(S_e, S_r , S_m) = \text{SDR}(S_e, S_r )−\text{SDR}(S_m, S_r )$ où $S_e$ est le signal isolé estimé, $S_r$ est le signal isolé de référence, et $S_m$ est le signal mix (original non séparé).
- Signal-to-Interference Ratio (SIR), 
- Signal-to-Artifact Ratio (SAR)

et une évaluation subjective menée par des humains.

##### Résultats

Sur la dataset iKala, moyennes :

| | U-Net | Baseline (U-Net sans skip-con.) | Chimera (ex-SOTA) |
| --- | --- | --- | --- |
| NSDR Vocal | **11.094** | 8.549 | 8.749 |
| NSDR Instrumental | **14.435** | 10.906 | 11.626 |
| SIR Vocal | **23.960** | 20.402 | 21.301 |
| SIR Instrumental | **21.832** | 14.304 | 20.481 |
| SAR Vocal | **17.715** | 15.481 | 15.642 |
| SAR Instrumental | **14.120** | 12.002 | 11.539 |

### Méthodes Forme d'onde
#### *D. Stoller  et Al., Wave-U-Net: A Multi-Scale Neural Network for End-to-end Audio Source Separation, ISMIR 2018*

##### Idée Generale
Adaptation du U-Net dans le domaine temporel. Leur modèle apprend directement à prédire les $K$ signaux sources qui font partie du signal mixé donné en entrée.

##### Architecture
![Wave-U-Net architecture](https://raw.githubusercontent.com/f90/Wave-U-Net/master/waveunet.png)

##### Évaluation
Ils utilisent seulement le SDR, et comparent avec le U-Net du domaine spectral, et une variante entraîné avec une loss L1 sur les magnitudes spectrales.

##### Résultats

| | M1 | M2 | M3 | M4 | M5 | M7 | U-Net | U-Net(a) |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| SDR Vocal | -0.12 | 0.05 | 0.31 | **0.65**| 0.55 | -0.23 | -0.66 | 0.51 |
| SDR Instrumental | 7.62 | 7.68 | 7.66 | **11.85** | 11.74 | 7.15 | 6.90 | 6.85 |

#### *Y. Luo et Al., TasNet: time-domain audio separation network for real-time, single-channel speech separation, ICASSP 2018* et *Y. Luo et Al.,  Conv-tasnet: surpassing ideal time–frequency magnitude masking for speech separation. IEEE/ACM Transactions on Audio, Speech, and Language Processing, 2019.*

##### Idée Générale
TasNet opère directement dans le domaine temporel à l'aide d'un modèle encodeur-décodeur.

- Encodeur : Le signal audio d'entrée est segmenté en courts morceaux (~5 ms) et transformé en une représentation non négative à l'aide d'une couche de convolution 1D.
- Module de séparation : Un réseau, basé sur des LSTM, génère des masques pour séparer les sources en estimant leur contribution respective dans la représentation encodée.
- Décodeur : Les masques sont appliqués aux représentations encodées pour reconstruire les signaux des différentes sources, directement dans le domaine temporel.

Conv-TasNet est une extension de TasNet. Il remplace les modules LSTM de TasNet par un réseau convolutif temporel, et est donc beaucoup plus rapide.

##### Architecture
**TasNet** :
![TasNet model architecture](https://ar5iv.labs.arxiv.org/html/1711.00541/assets/figures/flowchart_v2.png)
**Conv-TasNet** :
![Conv-TasNet model architecture](https://production-media.paperswithcode.com/methods/tasnet_4isUkmw.png)

##### Évaluation
Ils utilisent les métriques SI-SNR improvement (SI-SNRi : mesure la qualité de la séparation en comparant le rapport signal-bruit avant et après séparation, en prenant en compte l'invariance à l'échelle) and SDR improvement (SDRi : évalue la qualité de reconstruction en quantifiant la distorsion introduite dans les signaux séparés par rapport aux signaux de référence)

##### Résultats
| Méthode                 | SI-SNRi (dB)      | SDRi (dB)       |
|-------------------------|-------------------|-----------------|
| uPIT-LSTM               | -                 | 7.0             |
| TasNet-LSTM             | 7.7               | 8.0             |
| DPCL++                  | 10.8              | -               |
| DANet                   | 10.5              | -               |
| uPIT-BLSTM-ST           | -                 | 10.0            |
| TasNet-BLSTM            | 10.8              | 11.1            |
| Conv-TasNet             | **15.3**          | **15.6**        |



## Librairies utilisées

In [1]:
!pip install pystoi


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import matplotlib.pylab as plt
import scipy
import IPython.display as ipd
from pystoi import stoi
from scipy.signal import resample

## Visualisation des signaux

### Télécharger les données

In [ ]:
!mkdir data
# download denoising data
!wget -O data/denoising.zip https://cloud.leviia.com/s/ZnIy.3xerJBY8PDKNrBL/download?path=%2FProjets%2FAudio%2Fdenoising
# download voice_origin data
!wget -O data/voice_origin.zip https://cloud.leviia.com/s/ZnIy.3xerJBY8PDKNrBL/download?path=%2FProjets%2FAudio%2Fvoice_origin
# unzip all files
!unzip -qq -o data/denoising.zip -d data/
!unzip -qq -o data/voice_origin.zip -d data/
# unzip denoising folders
!unzip -qq data/denoising/test.zip -d data/denoising/
!echo "Unzipped denoising/test.zip"
!unzip -qq data/denoising/train.zip -d data/denoising/
!echo "Unzipped denoising/train.zip"
!unzip -qq data/denoising/train_small.zip -d data/denoising/
!echo "Unzipped denoising/train_small.zip"
# unzip voice_origin folders
!unzip -qq data/voice_origin/test.zip -d data/voice_origin/
!echo "Unzipped voice_origin/test.zip"
!unzip -qq data/voice_origin/train.zip -d data/voice_origin/
!echo "Unzipped voice_origin/train.zip"
!unzip -qq data/voice_origin/train_small.zip -d data/voice_origin/
!echo "Unzipped voice_origin/train_small.zip"
# remove all zip files
!rm data/denoising.zip
!rm data/voice_origin.zip
!rm data/denoising/test.zip
!rm data/denoising/train.zip
!rm data/denoising/train_small.zip
!rm data/voice_origin/test.zip
!rm data/voice_origin/train.zip
!rm data/voice_origin/train_small.zip
# done
!echo "Successfully downloaded and unzipped data."

--2024-12-09 14:33:12--  https://cloud.leviia.com/s/ZnIy.3xerJBY8PDKNrBL/download?path=%2FProjets%2FAudio%2Fdenoising
Resolving cloud.leviia.com (cloud.leviia.com)... 51.83.19.200, 46.105.50.201, 213.251.181.36
Connecting to cloud.leviia.com (cloud.leviia.com)|51.83.19.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘data/denoising.zip’

data/denoising.zip      [  <=>               ] 818.09M  30.3MB/s    in 28s     

2024-12-09 14:33:40 (29.6 MB/s) - ‘data/denoising.zip’ saved [857833302]

--2024-12-09 14:33:41--  https://cloud.leviia.com/s/ZnIy.3xerJBY8PDKNrBL/download?path=%2FProjets%2FAudio%2Fvoice_origin
Resolving cloud.leviia.com (cloud.leviia.com)... 51.83.19.200, 46.105.50.201, 213.251.181.36
Connecting to cloud.leviia.com (cloud.leviia.com)|51.83.19.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘data/voice_origin.zip’

data/voice_origin

In [ ]:
fs_noisy, audio_noisy = scipy.io.wavfile.read('./data/denoising/train_small/Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap01_0_.wav')
ipd.Audio('./data/denoising/train_small/Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap01_0_.wav')

In [ ]:
fs_clean, audio_clean = scipy.io.wavfile.read('./data/voice_origin/train_small/Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap01_0_.wav')
ipd.Audio('./data/voice_origin/train_small/Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap01_0_.wav')

In [ ]:
print(audio_clean.shape, audio_noisy.shape)
print(fs_clean, fs_noisy)
print(type(audio_clean))

(40000,) (80000,)
4000 8000
<class 'numpy.ndarray'>


In [ ]:
def resample_signal(signal, original_rate, target_rate):
    num_samples = int(len(signal) * target_rate / original_rate)
    resampled_signal = resample(signal, num_samples)
    return resampled_signal

Malheureusement le Kernel crash si on veut calculer le PESQ du signal, on se contente donc du STOI.

In [ ]:
def evaluate_model(noisy_signal, clean_signal, fs_noisy, fs_clean):
    if fs_noisy != fs_clean:
        clean_signal = resample_signal(clean_signal, fs_clean, fs_noisy)
    noisy_signal = noisy_signal / np.max(np.abs(noisy_signal))
    clean_signal = clean_signal / np.max(np.abs(clean_signal))
    stoi_value = stoi(clean_signal, noisy_signal, fs_noisy)
    return stoi_value

In [ ]:
stoi_value = evaluate_model(audio_noisy, audio_clean, fs_noisy, fs_clean)
print(stoi_value)

0.8455198761188121


## Implémentation d'une baseline : Filtre de Wiener

In [ ]:
from scipy.signal import welch

def wiener_filter_with_snr(audio, snr, fs):
    """
    Apply Wiener filtering to denoise an audio file using SNR.

    Parameters:
        audio (ndarray): Noisy signal with shape (40000,).
        snr (float): Signal-to-Noise Ratio in dB.
        fs (float): Sampling frequency.

    Returns:
        denoised_audio (ndarray): Denoised signal with shape (40000,).
    """

    # Estimate Signal Power (time-domain)
    signal_power = np.mean(audio**2)

    # Switch to linear-scaled SNR
    snr = 10**(snr / 10)

    # Compute Noise Power using SNR
    noise_power = signal_power / snr

    # Apply FFT to the signal
    fft_audio = np.fft.fft(audio)
    fft_freqs = np.fft.fftfreq(len(audio), d=1/fs)  # Frequency bins corresponding to the FFT

    # Compute PSD using Welch's method
    f, psd_audio = welch(audio, fs)
    print(f"PSD shape from Welch: {psd_audio.shape}")
    print(f"FFT frequency bins: {fft_freqs.shape}")

    # Interpolate the Wiener filter to match the FFT bins
    psd_audio_interp = np.interp(np.abs(fft_freqs[:len(psd_audio)]), f, psd_audio)
    # Make sure the length of psd_audio_interp matches the FFT bins
    psd_audio_interp = np.pad(psd_audio_interp, (0, len(audio) - len(psd_audio_interp)), mode='constant', constant_values=0)

    # Estimate clean signal PSD
    psd_clean = np.maximum(psd_audio_interp - noise_power, 1e-10)  # Avoid division by zero

    # Compute Wiener filter
    wiener_filter = psd_clean / (psd_clean + noise_power)
    print(f"Wiener filter shape: {wiener_filter.shape}")

    # Apply filter in frequency domain (element-wise multiplication)
    filtered_fft = fft_audio * wiener_filter

    # Transform back to time domain
    denoised_audio = np.fft.ifft(filtered_fft).real

    return denoised_audio

# Example usage
denoised_audio = wiener_filter_with_snr(audio_noisy, 20, fs_noisy)
ipd.Audio(denoised_audio, rate=fs_noisy)


PSD shape from Welch: (129,)
FFT frequency bins: (80000,)
Wiener filter shape: (80000,)


In [ ]:
stoi_value_denoised = evaluate_model(denoised_audio, audio_clean, fs_noisy, fs_clean)
print(stoi_value_denoised, stoi_value)

0.8455198750041509 0.8455198761188121


In [ ]:
class WarmupScheduler:
    def __init__(self, optimizer, warmup_iters, target_lr):
        self.optimizer = optimizer
        self.warmup_iters = warmup_iters
        self.target_lr = target_lr
        self.current_iter = 0

    def step(self):
        self.current_iter += 1
        if self.current_iter <= self.warmup_iters:
            # exponentially increase the learning rate
            warmup_lr = self.target_lr * (self.current_iter / self.warmup_iters) ** 2
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = warmup_lr

    def is_warmup_done(self):
        return self.current_iter >= self.warmup_iters

def train(
    model,
    train_dataloader,
    val_dataloader,
    device,
    optimizer,
    criterion,
    save_to,
    num_epochs=10,
    num_classes=6,
    base_lr=0.01,
    warmup=True,
    warmup_iters=50,
    plateau=True,
    lr_patience=5,
    lr_factor=0.5,
):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    learning_rates = []
    warmup_scheduler = WarmupScheduler(optimizer, warmup_iters, base_lr)
    plateau_scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=lr_patience, factor=lr_factor, verbose=True)

    model.to(device)

    pbar = tqdm(range(num_epochs))

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_accuracy = 0.0
        val_loss = 0.0
        val_accuracy = 0.0
        for signals, snr, labels_id in train_dataloader:
            signals, labels_id = signals.to(device), labels_id.to(device)
            optimizer.zero_grad()
            outputs = model(signals)
            loss = criterion(outputs, labels_id)
            loss.backward()
            optimizer.step()
            if not warmup_scheduler.is_warmup_done() and warmup:
                warmup_scheduler.step()
            train_loss += loss.item()
            train_accuracy += accuracy(outputs, labels_id, task='MULTICLASS', num_classes=num_classes).cpu().numpy()

        train_loss /= len(train_dataloader)
        train_accuracy /= len(train_dataloader)

        with torch.no_grad():
            model.eval()
            for signals, snr, labels_id in val_dataloader:
                signals, labels_id = signals.to(device), labels_id.to(device)
                outputs = model(signals)
                loss = criterion(outputs, labels_id)
                val_loss += loss.item()
                val_accuracy += accuracy(outputs, labels_id, task='MULTICLASS', num_classes=num_classes).cpu().numpy()

            val_loss /= len(val_dataloader)
            val_accuracy /= len(val_dataloader)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        pbar.update(1)
        pbar.set_description(
            f"train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}, train_accuracy: {train_accuracy:.4f}, val_accuracy: {val_accuracy:.4f}, lr: {optimizer.param_groups[0]['lr']}"
        )
        if (warmup_scheduler.is_warmup_done() or not warmup) and plateau:
            plateau_scheduler.step(val_accuracy)
        learning_rates.append(optimizer.param_groups[0]['lr'])
    pbar.close()

    torch.save(model, save_to)

    return train_losses, val_losses, train_accuracies, val_accuracies, learning_rates

In [ ]:
from Conv_TasNet_Pytorch.Conv_TasNet import ConvTasNet

# Load convTasNet with the official pytorch implementation from 
# https://github.com/JusperLee/Conv-TasNet/tree/master
conv_tasnet = ConvTasNet(
    N=512, # Number of ﬁlters in autoencoder
    L=16, # Length of the ﬁlters (in samples)
    B=128, # Number of channels in bottleneck and the residual paths’ 1 × 1-conv blocks
    H=512, # Number of channels in convolutional blocks
    P=3, # Kernel size in convolutional blocks
    X=8, # Number of convolutional blocks in each repeat
    R=3, # Number of repeats
    norm="gln",
    num_spks=2,
    activate="relu",
    causal=False
)